# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 10:26AM,256913,21,FF Preparation,"NBTY Global, Inc.",Released
1,Feb 8 2023 10:24AM,256912,10,0086363879,ISDIN Corporation,Released
2,Feb 8 2023 10:24AM,256912,10,0086363894,ISDIN Corporation,Released
3,Feb 8 2023 10:24AM,256911,16,SHL-9132360,"SHL Pharma, LLC",Released
4,Feb 8 2023 10:15AM,256910,10,SNAP34657391,"Snap Medical Industries, LLC",Released
5,Feb 8 2023 10:15AM,256910,10,SNAP34667060,"Snap Medical Industries, LLC",Released
6,Feb 8 2023 10:12AM,256909,10,130610,"Virtue Rx, LLC",Released
7,Feb 8 2023 10:11AM,256908,10,Clear-9131841,"ClearSpec, LLC",Released
8,Feb 8 2023 10:11AM,256908,10,Clear-9131843,"ClearSpec, LLC",Released
9,Feb 8 2023 10:11AM,256908,10,Clear-9131845,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,256909,Released,1
41,256910,Released,2
42,256911,Released,1
43,256912,Released,2
44,256913,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256909,NaN,NaN,1.0
256910,NaN,NaN,2.0
256911,NaN,NaN,1.0
256912,NaN,NaN,2.0
256913,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25.0,4.0,23.0
256821,0.0,3.0,1.0
256824,0.0,0.0,1.0
256826,0.0,0.0,113.0
256827,0.0,0.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25,4,23
256821,0,3,1
256824,0,0,1
256826,0,0,113
256827,0,0,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,4,23
1,256821,0,3,1
2,256824,0,0,1
3,256826,0,0,113
4,256827,0,0,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,4,23
1,256821,,3,1
2,256824,,,1
3,256826,,,113
4,256827,,,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc."
1,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation
3,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC"
4,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC"
6,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC"
7,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC"
11,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated"
13,Feb 8 2023 10:08AM,256906,10,MedStone Pharma LLC
14,Feb 8 2023 10:04AM,256826,15,"Person & Covey, Inc."
127,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",,,1
1,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,,,2
2,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",,,1
3,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",,,2
4,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",,,1
5,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC",,,4
6,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated",,,2
7,Feb 8 2023 10:08AM,256906,10,MedStone Pharma LLC,,,1
8,Feb 8 2023 10:04AM,256826,15,"Person & Covey, Inc.",,,113
9,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc.",,,34


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1,,
1,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,,
2,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1,,
3,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2,,
4,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",1,,
5,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC",4,,
6,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated",2,,
7,Feb 8 2023 10:08AM,256906,10,MedStone Pharma LLC,1,,
8,Feb 8 2023 10:04AM,256826,15,"Person & Covey, Inc.",113,,
9,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc.",34,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1,,
1,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,,
2,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1,,
3,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2,,
4,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2.0,NaN,NaN
2,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1.0,NaN,NaN
3,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
4,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 10:26AM,256913,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2.0,0.0,0.0
2,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1.0,0.0,0.0
3,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
4,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4880877,172.0,8.0,0.0
15,1284240,199.0,7.0,25.0
16,1027554,4.0,0.0,0.0
18,256892,0.0,1.0,0.0
19,1541133,27.0,20.0,0.0
21,513783,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4880877,172.0,8.0,0.0
1,15,1284240,199.0,7.0,25.0
2,16,1027554,4.0,0.0,0.0
3,18,256892,0.0,1.0,0.0
4,19,1541133,27.0,20.0,0.0
5,21,513783,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,172.0,8.0,0.0
1,15,199.0,7.0,25.0
2,16,4.0,0.0,0.0
3,18,0.0,1.0,0.0
4,19,27.0,20.0,0.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,172.0
1,15,Released,199.0
2,16,Released,4.0
3,18,Released,0.0
4,19,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21
Status,,,,,,
Completed,0.0,25.0,0.0,0.0,0.0,0.0
Executing,8.0,7.0,0.0,1.0,20.0,0.0
Released,172.0,199.0,4.0,0.0,27.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21
0,Completed,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,8.0,7.0,0.0,1.0,20.0,0.0
2,Released,172.0,199.0,4.0,0.0,27.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21
0,Completed,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,8.0,7.0,0.0,1.0,20.0,0.0
2,Released,172.0,199.0,4.0,0.0,27.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()